## import and download some necessary librairies

In [5]:
import pandas as pd
import numpy as np
import requests

import sys
!{sys.executable} -m pip install BeautifulSoup4
from bs4 import BeautifulSoup

import sys
!{sys.executable} -m pip install requests

import sys
!{sys.executable} -m pip install html5lib

import sys
!{sys.executable} -m pip install lxml

## scrape table of Toronto Postcode, Borough and Neighborhood from Wiki using BeautifulSoup

In [6]:
#create requests
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#create beautifulsoup
soup = BeautifulSoup(source, 'html5lib')

#scrape the table 
body = soup.find('body')
content = body.find('div', id = 'content')
bodyContent = content.find('div', id = 'bodyContent')
mw_content_text = bodyContent.find('div', id = 'mw-content-text')
mw_parser_output = mw_content_text.find('div', class_= 'mw-parser-output')
table_sorter = mw_parser_output.find('table', class_='wikitable sortable')

#pandas read html (result = list)
dfs = pd.read_html(str(table_sorter), header = 0)

#list to Dataframe
df = dfs[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Cleaning Dataframe

In [7]:
#change column header
header = ['PostalCode', 'Borough', 'Neighborhood']
df.columns = header

#raplace "Not assigned" with NaN for easy droping 
df.replace(to_replace = 'Not assigned', value  = pd.np.nan, inplace = True)

#drop rows if Borough = NaN
df.dropna(subset = ['Borough'],inplace = True)
df.reset_index(drop = True, inplace = True)

#replace all (Neighborhood = NaN) to its Borough name
index_null = df.index[np.nonzero(pd.isnull(df.values).any(1))]
for index in index_null:
	df.iloc[index, df.columns.get_loc('Neighborhood')] = df.iloc[index, df.columns.get_loc('Borough')]

#group Postalcode and add Neighborhood with the same Postalcode together
df_group = df.groupby(['PostalCode','Borough'], sort = False)['Neighborhood'].apply(', '.join).reset_index()

df_group.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
df_group.shape

(103, 3)

In [15]:
df_data = pd.read_csv('toronto_data.csv')

header1 = ['PostalCode', 'Latitude','Longitude']
df_data.columns = header1


df_toronto = df_group.set_index('PostalCode').join(df_data.set_index('PostalCode')).reset_index()

df_toronto.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
